In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Using temporal siries with TensorFlow. I think that could be better

In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df_amazon = pd.read_csv('../input/forest-fires-in-brazil/amazon.csv', encoding='latin1')

In [ ]:
df_amazon.head()

In [ ]:
dfa = df_amazon.groupby(['state', 'year'], as_index=False).agg({'number': 'sum'})

In [ ]:
dfa.head()

In [ ]:
base = dfa.iloc[:, 2].values

In [ ]:
plt.plot(base)

In [ ]:
periodos = 27
previsao_futura = 1 # horizonte das previsões

In [ ]:
X = base[0:(len(base) - (len(base) % periodos))]

In [ ]:
X_batches = X.reshape(-1, periodos, 1)

In [ ]:
y = base[1:(len(base) - (len(base) % periodos)) + previsao_futura]

In [ ]:
y_batches = y.reshape(-1, periodos, 1)

In [ ]:
X_teste = base[-(periodos + previsao_futura):]
X_teste = X_teste[:periodos]
X_teste = X_teste.reshape(-1, periodos, 1)

In [ ]:
y_teste = base[-(periodos):]
y_teste = y_teste.reshape(-1, periodos, 1)

In [ ]:
import tensorflow as tf

In [ ]:
tf.reset_default_graph()

In [ ]:
entradas = 1
neuronios_oculta = 100
neuronios_saida = 1

In [ ]:
xph = tf.placeholder(tf.float32, [None, periodos, entradas])
yph = tf.placeholder(tf.float32, [None, periodos, neuronios_saida])

In [ ]:
celula = tf.contrib.rnn.BasicRNNCell(num_units = neuronios_oculta, activation = tf.nn.relu)
celula = tf.contrib.rnn.OutputProjectionWrapper(celula, output_size = 1)

In [ ]:
saida_rnn, _ = tf.nn.dynamic_rnn(celula, xph, dtype = tf.float32)
erro = tf.losses.mean_squared_error(labels = yph, predictions = saida_rnn)
otimizador = tf.train.AdamOptimizer(learning_rate = 0.001)
treinamento = otimizador.minimize(erro)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoca in range(1000):
        _, custo = sess.run([treinamento, erro], feed_dict = {xph: X_batches, yph: y_batches})
        if epoca % 100 == 0:
            print(epoca + 1, ' erro: ', custo)
            
    previsoes = sess.run(saida_rnn, feed_dict = {xph: X_teste})

In [ ]:
import numpy as np

In [ ]:
y_teste.shape
y_teste2 = np.ravel(y_teste)

In [ ]:
previsoes2 = np.ravel(previsoes)

In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_teste2, previsoes2)
mae

In [ ]:
plt.plot(y_teste2, markersize = 10, label = 'Valor real')
plt.plot(previsoes2, label = 'Previsões')
plt.legend()